# core

> Fill in a module description here

In [69]:
#| default_exp creds

In [70]:
#| hide
from nbdev.showdoc import *


In [71]:
#| exporti
from typing import Union
from enum import Enum
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [72]:
#| exporti

import os
import json

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

In [73]:
#| export

class GenerateTokenFile_RequiresClientInfo(Exception):
    def __init__(self):
        super().__init__("must pass client_secret_path or client_secret_info, this information can be downloaded from https://console.cloud.google.com/apis/credentials/oauthclient")


def generate_token_from_oauth_with_client_info(token_file_path: str = 'env_token.json',
                        client_secret_path: str = None,
                        client_secret_info: dict = None,
                        scopes: [str] = ['https://www.googleapis.com/auth/drive.metadata.readonly',
                                         'https://www.googleapis.com/auth/drive.readonly']
                        ):

    """the client information, downloaded from https://console.cloud.google.com/apis/credentials/oauthclient will be used to generate a token file which will be used to authenticate API requests
    
this function accepts the client_secret_path (a local .json file) or the client_secret_info dictionary
then initiates the ouath flow for exchanging a token file.

The file env_token.json stores the user's access and refresh tokens, and is
created automatically when the authorization flow completes for the first time.

    """

    if not client_secret_info and not client_secret_path:
        raise GenerateTokenFile_RequiresClientInfo()

    if client_secret_path:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_path, scopes)
    
    elif client_secret_info:
        flow = InstalledAppFlow.from_client_config(client_secret_info, scopes)
    
    token = flow.run_local_server(port= 0,)

    # Save the credentials for the next run
    with open(token_file_path, 'w') as f:
        f.write(token.to_json())
    
    return token


In [74]:
# in this example we assume the client secret information is loaded into the environment as GDOC_KEY using a .env file
# we must generate an env_token.json file for authenticating queries

from dotenv import load_dotenv

load_dotenv('.env')

client_secret_info=json.loads(os.environ['GDOC_KEY'])

if not os.path.exists('env_token.json'):
    generate_token_from_oauth_with_client_info(client_secret_info = client_secret_info)


#### sample implementation of get_creds

In [75]:
#| export

def generate_creds_object(
        token_file_path: str = None,
        token_info: dict = None,
        client_secret_path: str = None,
        client_secret_info: dict = None):
    """generate creds object from token file"""
    
    creds = None

    if token_file_path:
        creds = Credentials.from_authorized_user_file(token_file_path)

    if token_info:
        creds = Credentials.from_authorized_user_info(token_info)

    if creds and creds.valid:
        return creds

    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
        return creds

    # If there are no (valid) credentials available, let the user log in.
    return generate_token_from_oauth_with_client_info(token_file_path=token_file_path or 'env_token.json',
                                                      client_secret_path=client_secret_path,
                                                      client_secret_info=client_secret_info)


In [76]:
from dotenv import load_dotenv

load_dotenv('.env')

client_secret_info=json.loads(os.environ['GDOC_KEY'])

token_info = json.loads(os.environ['GDOC_TOKEN'])

generate_creds_object(client_secret_info=client_secret_info, token_info = token_info)

In [77]:
#| hide
import nbdev; nbdev.nbdev_export()